In [1]:
from IPython.core.debugger import Tracer
import tensorflow as tf
import keras
import keras.losses
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, BatchNormalization, Concatenate
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential, load_model, Model
from keras.optimizers import Adam, Nadam
from keras.callbacks import EarlyStopping, Callback
import keras.backend as K

from scipy import io
import scipy as sp
import numpy as np
# import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import host_subplot
import mpl_toolkits.axisartist as AA
import random
import csv
import os
import datetime
import socket
import socketserver
import time
import threading
from IPython.display import clear_output

np.set_printoptions(precision=10, suppress=True)
def my_mape(y_true, y_pred):
    return K.sum((K.abs(y_pred - y_true))/(K.abs(y_true)+0.001))
keras.losses.my_mape = my_mape


Q_max = 1
Q_min = -1
def Q_normalize(Q_num):
    return (Q_num - Q_min)/(Q_max - Q_min)

def Q_denormalize(X_pred_nor):
    res = X_pred_nor.copy()
#     Freq = res[:,2:20]
#     res[:,2:20] = np.power(10, Freq)
    Q_num = res[:,13:]
    Q_num = Q_num*(Q_max - Q_min)+Q_min
    res[:,13:] = Q_num
    return res

class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True
            
sys_eq = np.zeros((4, 12))
sys_eq[0] = [0,5.37160658715533e-06,1.01642079247645e-05,-3.15029114017033e-05,1.61652283511178e-05,0, 
             1,-4.08482411682325,6.61934635300307,-5.31658479192205,2.11560994953312,-0.333547393790890]

sys_eq[1] = [0,3.07689297967083e-05,-5.95779947442932e-05,2.88360987927538e-05,0,0, 
             1,-3.81134086427335,5.47923789803252,-3.55757241426427,0.922956462040706,-0.0332810815356037]

sys_eq[2] = [0,1.94643500831292e-05,-4.19161004462964e-05,3.02909548271302e-05,-7.40278369851218e-06,0, 
             1,-4.44700230387150,7.99839085487672,-7.28252340101705,3.36004265435182,-0.628907804339994]

sys_eq[3] = [0,3.46455304844684e-05,-6.64244171183570e-05,3.18123832826486e-05,0,0, 
             1,-3.60556002424451,4.67096932344118,-2.36455301822877,0.138605402571258,0.160538316460838]
sys_eq[:,:6] = sys_eq[:,:6]*100000

Using TensorFlow backend.


In [2]:
# Load datas
poles = []
Qnums = []
maxErr = []
rmsErr = []

directory_name = './MATLAB/data/0611/'
files_name = os.listdir(directory_name)
print(directory_name, len(files_name), 'files')
for file in files_name:
    filein = np.genfromtxt(directory_name + file, delimiter=',')
    poles.append(filein[:, 0:1])
    Qnums.append(filein[:, 1:16])
    maxErr.append(filein[:, 17:18])
    rmsErr.append(filein[:, 18:])
    
poles = np.array(poles)
Qnums = np.array(Qnums)
maxErr = np.array(maxErr)
rmsErr = np.array(maxErr)
sysEQ = np.zeros((poles.shape[0], poles.shape[1], sys_eq.shape[1]))
sysEQ[:, :, :] = sys_eq.reshape(4, -1, 12)
Q_max = Qnums.max(axis = (0, 1))
Q_min = Qnums.min(axis = (0, 1))
print(poles.shape, Qnums.shape, sysEQ.shape, maxErr.shape, rmsErr.shape)

./MATLAB/data/0611/ 4 files
(4, 300, 1) (4, 300, 15) (4, 300, 12) (4, 300, 1) (4, 300, 1)


In [4]:
Qnums[0,::3].shape

(100, 15)

In [6]:
with open('./MATLAB/data/Generate/0611_2/aa.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(Qnums[0,::3])
with open('./MATLAB/data/Generate/0611_2/bb.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(Qnums[1,::3])    


In [ ]:
fig = plt.figure(figsize=(15, 16))
for i in range(4):
    ax1 = host_subplot(42*10 + i*2+1, axes_class=AA.Axes)
    ax1.plot(maxErr[i], alpha = 0.5)
    ax1 = host_subplot(42*10 + i*2+2, axes_class=AA.Axes)
    for j in range(15):
        ax1.plot(Qnums[i, :, j], alpha = 0.5)
plt.show()


In [ ]:
with open('./MATLAB/data/Generate/0611_2/temp_0.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(np.dstack((sysEQ, poles, Qnums)).reshape(-1, 28))
with open('./MATLAB/data/Generate/0611_2/temp_error_0.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(np.dstack((maxErr, rmsErr)).reshape(-1, 2))    


In [ ]:
Y_in_ = []
Y_out_ = []
indices_ = []
# data_idx = [0, 1, 2, 5, 6, 7, 8, 12, 18, 24]
data_idx = range(1)
total = 0
for idx in data_idx:
    sys = np.genfromtxt('./MATLAB/data/Generate/0611_2/temp_{}.csv'.format(idx), delimiter=',')[:,:12]
    p = np.genfromtxt('./MATLAB/data/Generate/0611_2/temp_{}.csv'.format(idx), delimiter=',')[:,12:13]
    q = Q_normalize(np.genfromtxt('./MATLAB/data/Generate/0611_2/temp_{}.csv'.format(idx), delimiter=',')[:,13:])
    Y_in_.append(np.hstack((sys, p, q)))
    Y_out_.append(np.genfromtxt('./MATLAB/data/Generate/0611_2/temp_error_{}.csv'.format(idx), delimiter=',')[:,0]*1000)
    print(Y_in_[-1].shape, Y_out_[-1].shape)
    indices_.append(np.arange(Y_in_[-1].shape[0]) + total)
    total += Y_in_[-1].shape[0]
    np.random.shuffle(indices_[-1])


In [ ]:
q

In [ ]:
X_in = Y_in_[0][:,:13]
X_out = Y_in_[0][:,13:]
Y_in = Y_in_[0]
Y_out = Y_out_[0]
for i in range(1,len(Y_out_)):
    Y_in = np.append(Y_in, Y_in_[i], axis = 0)
    Y_out = np.append(Y_out, Y_out_[i], axis = 0)
Y_in = np.delete(Y_in, np.where(Y_out != Y_out), axis = 0)
Y_out = np.delete(Y_out, np.where(Y_out != Y_out), axis = 0)
print(X_in.shape, X_out.shape, Y_in.shape, Y_out.shape)
indices = np.arange(Y_in.shape[0])
np.random.shuffle(indices)

fig = plt.figure(figsize=(15, 16))
ax1 = host_subplot(211, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(Y_in_[0])[:,i+13], alpha = 0.5)
    ax1.set_title('Original Q num')
    

ax1 = host_subplot(212, axes_class=AA.Axes)
ax1.plot(Y_out_[0], alpha = 0.5)
ax1.set_title('Original Err')
# ax1.set_ylim(0,10)
plt.show()


# fig = plt.figure(figsize=(15, 16))
# ax1 = host_subplot(211, axes_class=AA.Axes)
# for i in range(15):
#     ax1.plot(Q_denormalize(Y_in)[:,i+13], alpha = 0.5)
#     ax1.set_title('Original Q num')

# ax1 = host_subplot(212, axes_class=AA.Axes)
# ax1.plot(Y_out, alpha = 0.5)
# ax1.set_title('Original Err')
# # ax1.set_ylim(0,50)
# plt.show()


In [ ]:
keras.backend.clear_session()

In [ ]:
def __discriminator():
    """ Declare discriminator """
    model = Sequential()
    model.add(Dense(units=32, input_dim=Y_in.shape[1], kernel_initializer='normal'))
#     model.add(Dropout(0.1))
#     model.add(BatchNormalization())
    model.add(Dense(units=64, kernel_initializer='normal', activation='sigmoid'))
#     model.add(BatchNormalization(momentum=0.8))
    model.add(Dense(units=16, activation='sigmoid'))
#     model.add(Dropout(0.1))
    model.add(Dense(units=1))
    model.name = "D"
    return model

In [ ]:
def __generator():
    """ Declare generator """
    G_in = Input(shape = (X_in.shape[1],))
    L1 = Dense(units=64, kernel_initializer='normal')(G_in)
#     L1 = Dropout(0.1)(L1)
    L1 = Dense(units = 128)(L1)
#     L1 = Dropout(0.1)(L1)
#     L1 = BatchNormalization()(L1)
    L1 = Dense(units = 64, activation='sigmoid')(L1)
#     L1 = Dropout(0.1)(L1)
#     L1 = BatchNormalization()(L1)
    L1 = Dense(units = 32, activation='sigmoid')(L1)
    G_out = Dense(units = X_out.shape[1])(L1)
    G_out = Concatenate(axis = -1)([G_in, G_out])
    model = Model(inputs = G_in, outputs = G_out)
    model.name = "G"
    return model

In [ ]:
def __stacked_generator_discriminator(self_G, self_D):
    self_D.trainable = False
    model = Sequential()
    model.add(self_G)
    model.add(self_D)
    model.name = "GD"
    return model

In [ ]:
self_D = __discriminator()
self_D.summary()

In [ ]:
self_G = __generator()
self_G.summary()

In [ ]:
self_stacked_GD = __stacked_generator_discriminator(self_G, self_D)
self_stacked_GD.summary()

In [ ]:
logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
self_D.trainable = True
self_D.compile(loss = 'mae', optimizer = Nadam(lr=0.002))
train_history_D = self_D.fit(x=Y_in, y=Y_out, 
                             validation_split=0, epochs=100000, batch_size=64, verbose=1, 
                             callbacks = [tensorboard_callback, 
                                          EarlyStopping(monitor = 'loss', patience = 5000, verbose = 1, mode = 'auto'),
                                          EarlyStoppingByLossVal(monitor='loss', value=0.01, verbose=1)])

In [ ]:
self_G.compile(loss='mae', optimizer=Nadam(lr=0.0001))
train_history_G = self_G.fit(x=X_in, y=Y_in,
                               validation_split=0.2, epochs=100000, batch_size=1024, verbose=1, 
                               callbacks = [tensorboard_callback, EarlyStopping(monitor = 'val_loss', patience = 5000, verbose = 1, mode = 'auto')])

In [ ]:
mislabled = np.zeros((Y_in_[0].shape[0], 1))
self_D.trainable = False
self_stacked_GD.compile(loss='mape', optimizer=Nadam(lr=0.0001))
train_history_GD = self_stacked_GD.fit(x=X_in[indices_[0]], y=Y_out[indices_[0]], 
# train_history_GD = self_stacked_GD.fit(x=X_in, y=mislabled,
                                       validation_split=0.1, epochs=1000000, batch_size=64, verbose=1, 
                                       callbacks = [tensorboard_callback, EarlyStopping(monitor = 'loss', patience = 1000, verbose = 1, mode = 'auto')])

In [ ]:
import warnings

In [ ]:
self_G = load_model('./models/Simulator_DesignF2Qnum_0601_4.h5') 
self_G.name = "G"
self_G.compile(loss=my_mape, optimizer=Adam(lr=0.0000001, beta_1=0.5, decay=8e-8))
self_G.summary()
self_D = load_model('./models/Simulator_Qnum2Errmax_0601_4.h5')
self_D.name = "D"
self_D.compile(loss='mape', optimizer=Adam(lr=0.000001, beta_1=0.5, decay=8e-8))
self_D.summary()

self_stacked_GD = __stacked_generator_discriminator(self_G, self_D)
self_stacked_GD.compile(loss='mae', optimizer=Adam(lr=0.0002, beta_1=0.5, decay=8e-8))
self_stacked_GD.summary()

In [ ]:
self_G.save('./models/Simulator_DesignF2Qnum_0608_2.h5')
self_D.save('./models/Simulator_Qnum2Errmax_0608_2.h5')
self_stacked_GD.save('./models/Stacked_GAN_0608_2.h5')

In [ ]:
serverSocket =socket.socket(socket.AF_INET,socket.SOCK_STREAM)
serverSocket.bind (('127.0.0.1',50000))
serverSocket.listen(1)

In [ ]:
train_history_D = []
train_history_GD = []
for idx in range(20):

    X_pred = self_G.predict(X_in)
    with open('./MATLAB/data/Generate/temp.csv', 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(Q_denormalize(X_pred))
    print("start matlab")
    os.system(r"matlab -r \"run('C:\Users\dalek669528\Desktop\CNC_proj\MATLAB\Simulation_FixedQ_Ccode\X_main_fixedQ_Ccode_func.m')\"")
    print("open socket")
    connectionSocket, addr = serverSocket.accept()
    with open('./MATLAB/data/Generate/check_{}.csv'.format(idx), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(Q_denormalize(X_pred))
    self_G.save('./models/checkpoint/G_{}.h5'.format(idx))
    self_D.save('./models/checkpoint/D_{}.h5'.format(idx))
    self_stacked_GD.save('./models/checkpoint/StackGD_{}.h5'.format(idx))
    
    self_D.trainable = True
    self_D.compile(loss = 'mae', optimizer = Nadam(lr=0.00001))
    train_history_D = self_D.fit(x=Y_in, y=Y_out, 
                                 validation_split=0, epochs=100000, batch_size=64, verbose=1, 
                                 callbacks = [tensorboard_callback, 
                                              EarlyStopping(monitor = 'loss', patience = 2000+idx*1000, verbose = 1, mode = 'auto')])

    mislabled = np.zeros((Y_in_[0].shape[0], 1))
#     self_G = __generator()
#     self_stacked_GD = __stacked_generator_discriminator(self_G, self_D)
    self_D.trainable = False
    self_stacked_GD.compile(loss='mae', optimizer=Nadam(lr=0.00001))
    train_history_GD = self_stacked_GD.fit(x=X_in[indices_[0]], y=Y_out[indices_[0]], 
    # train_history_GD = self_stacked_GD.fit(x=X_in[:3750], y=mislabled,
                                           validation_split=0.1, epochs=1000000, batch_size=64, verbose=1, 
                                           callbacks = [tensorboard_callback, EarlyStopping(monitor = 'loss', patience = 1000, verbose = 1, mode = 'auto')])
    
#     connectionSocket, addr = serverSocket.accept()
    sentence = connectionSocket.recv(1024)
    print(sentence.decode())
    connectionSocket.close() 
    with open('./MATLAB/data/Generate/0610/temp_{}.csv'.format(len(Y_out_)), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=','))
    with open('./MATLAB/data/Generate/0610/temp_error_{}.csv'.format(len(Y_out_)), 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerows(np.genfromtxt('./MATLAB/data/Generate/temp_error.csv', delimiter=','))    
    p_LQ = np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,:1]
    #     Freq = np.log10(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,2:20])
    Q = Q_normalize(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,1:])
    Y_in_resimulate = np.hstack((p_LQ, Q))
    Y_out_resimulate = np.genfromtxt('./MATLAB/data/Generate/temp_error.csv', delimiter=',')[:,0]*1000
    Y_in_.append(Y_in_resimulate)
    Y_out_.append(Y_out_resimulate)
    Y_in = np.append(Y_in, Y_in_resimulate, axis=0)
    Y_out = np.append(Y_out, Y_out_resimulate)
    Y_in = np.delete(Y_in, np.where(Y_out != Y_out), axis = 0)
    Y_out = np.delete(Y_out, np.where(Y_out != Y_out), axis = 0)
    print(Y_in_[-1].shape, Y_out_[-1].shape, len(Y_in))
    indices = np.arange(Y_in.shape[0])
    np.random.shuffle(indices)

In [ ]:
with open('./MATLAB/data/Generate/0601/temp_0601_{}.csv'.format(len(Y_out_)), 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=','))
with open('./MATLAB/data/Generate/0601/temp_error_0601_{}.csv'.format(len(Y_out_)), 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(np.genfromtxt('./MATLAB/data/Generate/temp_error.csv', delimiter=','))    
p_LQ = np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,:1]
#     Freq = np.log10(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,2:20])
Q = Q_normalize(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,1:])
Y_in_resimulate = np.hstack((p_LQ, Q))
Y_out_resimulate = np.genfromtxt('./MATLAB/data/Generate/temp_error.csv', delimiter=',')[:,0]*1000
Y_in_.append(Y_in_resimulate)
Y_out_.append(Y_out_resimulate)
Y_in = np.append(Y_in, Y_in_resimulate, axis=0)
Y_out = np.append(Y_out, Y_out_resimulate)
Y_in = np.delete(Y_in, np.where(Y_out != Y_out), axis = 0)
Y_out = np.delete(Y_out, np.where(Y_out != Y_out), axis = 0)
print(Y_in_[-1].shape, Y_out_[-1].shape, len(Y_in))
indices = np.arange(Y_in.shape[0])
np.random.shuffle(indices)

In [ ]:
Y_in_.append(Y_in_resimulate)
Y_out_.append(Y_out_resimulate)
Y_in = np.append(Y_in, Y_in_resimulate, axis=0)
Y_out = np.append(Y_out, Y_out_resimulate)
print(Y_in_[-1].shape, Y_out_[-1].shape, len(Y_in))

In [ ]:
p_LQ = np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,:1]
#     Freq = np.log10(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,2:20])
Q = Q_normalize(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',')[:,1:])
Y_in_resimulate = np.hstack((p_LQ, Q))
Y_out_resimulate = np.genfromtxt('./MATLAB/data/Generate/temp_error.csv', delimiter=',')[:,0]*1000
Y_in_.append(Y_in_resimulate)
Y_out_.append(Y_out_resimulate)
Y_in = np.append(Y_in, Y_in_resimulate, axis=0)
Y_out = np.append(Y_out, Y_out_resimulate)
Y_in = np.delete(Y_in, np.where(Y_out != Y_out), axis = 0)
Y_out = np.delete(Y_out, np.where(Y_out != Y_out), axis = 0)
print(Y_in_[-1].shape, Y_out_[-1].shape, len(Y_in))
indices = np.arange(Y_in.shape[0])
np.random.shuffle(indices)

In [ ]:
np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=',').shape

In [ ]:
# with open('./MATLAB/data/Generate/temp_0430_{}.csv'.format(len(Y_out_)), 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerows(np.genfromtxt('./MATLAB/data/Generate/temp.csv', delimiter=','))
# with open('./MATLAB/data/Generate/temp_error_0430_{}.csv'.format(len(Y_out_)), 'w', newline='') as csvfile:
#     writer = csv.writer(csvfile)
#     writer.writerows(np.genfromtxt('./MATLAB/data/Generate/temp_error.csv', delimiter=','))

In [ ]:
X_pred = self_G.predict(X_in)
Y_pred = self_stacked_GD.predict(X_in)
Y_pred_D = self_D.predict(Y_in_[-2])
fig = plt.figure(figsize=(15, 16))
ax1 = host_subplot(211, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(Y_in_[-2])[:,i+20], alpha = 0.5)
    ax1.set_title('Original Q num')
ax1 = host_subplot(212, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(X_pred)[:,i+20], alpha = 0.5)
    ax1.set_title('Original Q num')
plt.show()

fig = plt.figure(figsize=(15, 20))
ax1 = host_subplot(311, axes_class=AA.Axes)
ax1.plot(Y_out_[-2], alpha = 0.5)
ax1.set_title('Original Err')
ax1 = host_subplot(312, axes_class=AA.Axes)
ax1.plot(Y_pred, alpha = 0.5)
ax1.set_title('Original Err')
ax1 = host_subplot(313, axes_class=AA.Axes)
ax1.plot(Y_pred_D, alpha = 0.5)
ax1.set_title('Original Err')
plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 16))
ax1 = host_subplot(211, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(Y_in_[-1])[:,i+20], alpha = 0.5)
    ax1.set_title('Original Q num')

ax1 = host_subplot(212, axes_class=AA.Axes)
ax1.plot(Y_out_[-1], alpha = 0.5)
ax1.set_title('Original Err')
plt.show()


In [ ]:
Q_denormalize(X_pred)[1300]

In [ ]:
X_pred = self_G.predict(X_in)
Y_pred = self_stacked_GD.predict(X_in)
Y_pred_D = self_D.predict(Y_in)
fig = plt.figure(figsize=(15, 16))
ax1 = host_subplot(211, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(Y_in_[0])[:,i+13], alpha = 0.5)
    ax1.set_title('Original Q num')
# plt.show()
# fig = plt.figure(figsize=(15, 8))
ax1 = host_subplot(212, axes_class=AA.Axes)
for i in range(15):
    ax1.plot(Q_denormalize(X_pred)[:,i+13], alpha = 0.8)
    ax1.set_title('Predict Q num')
plt.show()


fig = plt.figure(figsize=(15, 8))
ax1 = host_subplot(111, axes_class=AA.Axes)
ax1.plot(Y_out_[0], alpha = 1, label = "Original error")
ax1.plot(Y_pred, alpha = 0.5, label = "Predict error")
ax1.set_title('max error')
ax1.legend()
# ax1.set_ylim(0, 1)
plt.show()

fig = plt.figure(figsize=(15, 8))
ax1 = host_subplot(111, axes_class=AA.Axes)
ax1.plot(Y_out, alpha = 0.3, label = "Original error")
ax1.plot(Y_pred_D, alpha = 0.5, label = "Predict error")
ax1.set_title('max error')
ax1.legend()
# ax1.set_ylim(0, 1)
plt.show()

# fig = plt.figure(figsize=(15, 8))
# ax1 = host_subplot(111, axes_class=AA.Axes)
# ax1.plot(Y_out, alpha = 0.5, label = "Original error")
# ax1.plot(Y_pred_D, alpha = 0.5, label = "Predict error")
# ax1.set_title('max error')
# ax1.legend()
# ax1.set_ylim(0, 50)
# plt.show()

In [ ]:
print(tf.__version__)

In [ ]:
Y_pred_D = self_D.predict(Y_in)
fig = plt.figure(figsize=(15, 8))
ax1 = host_subplot(111, axes_class=AA.Axes)
ax1.plot(Y_out[:], alpha = 1, label = "Original error")
ax1.plot(Y_pred_D[:], alpha = 0.3, label = "Predict error")
ax1.set_title('max error')
ax1.legend()

In [ ]:
# Y_test = self_D.predict(Y_in)
fig = plt.figure(figsize=(15, 16))
ax1 = host_subplot(211, axes_class=AA.Axes)
ax1.plot(Y_out_, alpha = 0.9)
# ax1.plot(Y_test, alpha = 0.5)
# ax1 = host_subplot(212, axes_class=AA.Axes)
# ax1.plot(Y_out_3[:200], alpha = 0.9)
# ax1.plot(Y_test[:200], alpha = 0.5)
plt.show()

In [ ]:
for idx in range(len(Y_out_)):
    fig = plt.figure(figsize=(10, 8))
    ax1 = host_subplot(211, axes_class=AA.Axes)
    for i in range(15):
        ax1.plot(Q_denormalize(Y_in_[idx])[:,i+20], alpha = 0.5)
    ax1.set_title(idx)
#     ax1.set_xticklabels(['','3', '3.5', '4', '4.5', '5', '5.5', '6', '6.5'], fontsize = 30)
#     plt.show()

#     fig = plt.figure(figsize=(7, 5))
    ax1 = host_subplot(212, axes_class=AA.Axes)
    ax1.plot(Y_out_[idx], alpha = 0.5)
    # ax1.set_title('Original Err')
#     ax1.set_xticklabels(['','3', '3.5', '4', '4.5', '5', '5.5', '6', '6.5'], fontsize = 30)
    ax1.set_ylim(0,25)
    plt.show()

In [ ]:
in_ = np.genfromtxt('./MATLAB/data/Generate/0601/temp_0601_2.csv', delimiter=',')
out_ = np.genfromtxt('./MATLAB/data/Generate/0601/temp_error_0601_2.csv', delimiter=',')
print(in_.shape, out_.shape)

In [ ]:
with open('./MATLAB/data/Generate//0601/temp_0601_2.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(in_)
with open('./MATLAB/data/Generate/0601/temp_error_0601_2.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(out_)